## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Enable gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\TracieFauth\Documents\Data Bootcamp\Week 6 APIs\world_weather_analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Current Description
0,Leer,SS,79.50,66,96,4.27,8.3018,30.1418,overcast clouds
1,Port Hardy,CA,51.80,81,75,8.05,50.6996,-127.4199,broken clouds
2,Uvalde,US,97.32,28,0,13.27,29.3669,-99.7670,clear sky
3,Ushuaia,AR,44.22,81,20,19.57,-54.8000,-68.3000,few clouds
4,Armidale,AU,43.63,95,79,8.43,-30.5167,151.6500,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Current Description
0,Leer,SS,79.50,66,96,4.27,8.3018,30.1418,overcast clouds
1,Port Hardy,CA,51.80,81,75,8.05,50.6996,-127.4199,broken clouds
3,Ushuaia,AR,44.22,81,20,19.57,-54.8000,-68.3000,few clouds
4,Armidale,AU,43.63,95,79,8.43,-30.5167,151.6500,broken clouds
5,Mayumba,GA,78.40,92,66,6.06,-3.4320,10.6554,broken clouds
7,Portland,US,56.86,62,75,8.01,45.5234,-122.6762,broken clouds
8,Port Alfred,ZA,62.15,78,14,4.99,-33.5906,26.8910,few clouds
10,San Jerónimo,PE,63.01,45,54,1.05,-11.9472,-75.2829,broken clouds
11,Mataura,NZ,49.48,96,96,1.07,-46.1927,168.8643,overcast clouds
13,Sibu,MY,77.31,94,40,0.00,2.3000,111.8167,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Latitude               0
Longitude              0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Latitude", "Longitude"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Latitude,Longitude,Hotel Name
0,Leer,SS,79.50,overcast clouds,8.3018,30.1418,
1,Port Hardy,CA,51.80,broken clouds,50.6996,-127.4199,
3,Ushuaia,AR,44.22,few clouds,-54.8000,-68.3000,
4,Armidale,AU,43.63,broken clouds,-30.5167,151.6500,
5,Mayumba,GA,78.40,broken clouds,-3.4320,10.6554,
7,Portland,US,56.86,broken clouds,45.5234,-122.6762,
8,Port Alfred,ZA,62.15,few clouds,-33.5906,26.8910,
10,San Jerónimo,PE,63.01,broken clouds,-11.9472,-75.2829,
11,Mataura,NZ,49.48,overcast clouds,-46.1927,168.8643,
13,Sibu,MY,77.31,scattered clouds,2.3000,111.8167,


In [9]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get latitude and longitude from DataFrame
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

# Make request and get the JSON data from the search.
hotels

Hotel not found... skipping.
Hotel not found... skipping.


{'html_attributions': [],
 'next_page_token': 'Aap_uEAdrtmq70m__Ok3bitkMPKRen_Hco3_ZPlsDnq8A9sfqt5IIdB1TO54M14ORnLQd8ACbwGbTQIc_jpRnqXWeav3SjPZcSAfwBf0QEFnKyAvhngf4MBUy92nPry2PScQQ1LAmyzccsYoXdhG43o7AJ8P_9oAEdDVva3vCc_UOuW4VPOJ-oCigA40w7l4D1FWSxQfAGU3WqsSOlwQC0VC-Ty_6rvEEmNGuvEOgtIwlU1WRwlx1KhelSCzaFwl88z2zK_GlHrdzOO0ow9oelMyeU-qQs49vRW3cL7hwQLkR7h_s70b2_sgKLDMzT9uu5U0e9LbEqlIiPbm02DCpGUqQ-5ehXpctVOJxSj6SArxpg9Rdhi-oY8FElSzbNn1XBogcpFXB3_hLprwwuv8IPNaM-7wrRVgS5A51tBmZlc3UxGQOirt0qreu_jw',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -33.64639289999999, 'lng': 115.3486762},
    'viewport': {'northeast': {'lat': -33.6450482697085,
      'lng': 115.3498651802915},
     'southwest': {'lat': -33.6477462302915, 'lng': 115.3471672197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinl

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# Export the City_Data into a csv
clean_hotel_df.to_csv("WeahterPy_Vacation.csv", index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_w1 = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_w1.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Latitude", "Longitude"]]
max_temp = clean_hotel_df["Max Temp"]

In [18]:
# Heatmap of temperature

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=[max(temp, 0) for temp in max_temp],
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
#Marker variable
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the heatmap layer.
fig.add_layer(heat_layer)
# Add marker layer
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))